In [ ]:
from importlib import reload
import config.conf as conf
import geopandas as gpd
from scripts import preprocessing as pre

In [ ]:
conf = reload(conf)
config = conf.Configuration()

In [ ]:
# Read the training area and training polygons
trainingArea = gpd.read_file(config.training_area)
trainingPolygon = gpd.read_file(config.training_polygon)
assert trainingPolygon.crs == trainingArea.crs

In [ ]:
len(trainingPolygon), len(trainingArea)

In [ ]:
# Assign serial IDs to training areas and polygons
trainingArea["id"] = range(len(trainingArea))
trainingPolygon["id"] = range(len(trainingPolygon))

In [ ]:
# areasWithPolygons contains the object polygons and weighted boundaries for each area!
areas_with_polygons = pre.dividePolygonsInTrainingAreas(trainingPolygon, trainingArea)

In [ ]:
conf = reload(conf)
config = conf.Configuration()
input_images = pre.read_input_images(
    config.train_image_dir,
    config.raw_image_file_type,
    config.raw_image_suffix,
)
print(f"Found a total of {len(input_images)} pair of raw image(s) to process!")

In [ ]:
pre = reload(pre)

In [ ]:
overlaps = pre.findOverlap(input_images,areas_with_polygons)

In [ ]:
pre.create_annotations(overlaps, areas_with_polygons)

### Check image dimensions.

Check input image dimensions to determine a proper shape for the NN inputs. This has to be done due to different sizes in the training areas.

In [ ]:
dims = {img.name: {
    "w":rio.open(img).profile["width"],
    "h":rio.open(img).profile["height"],
} for img in list(config.image_dir.glob("*png"))}

In [ ]:
ws = min([dim["w"] for dim in dims.values()])
hs = min([dim["h"] for dim in dims.values()])
(ws, hs)

## Check images

In [ ]:
from config import conf
import rasterio as rio
from shapely.geometry import box
from core.frame_info import image_normalize
from importlib import reload
conf = reload(conf)
config = conf.Configuration()

In [ ]:
test_img, id_ = ['afk_mean_composite_202', 2]
# test_img, id_ = ['afk_mean_composite_area3', 10]

In [ ]:
test_image = list(config.train_image_dir.glob(f"*{test_img}*.tif"))[0]
test_image

In [ ]:
img = rio.open(test_image)
profile = img.profile

In [ ]:
bboxArea = box(*areas_with_polygons[id_]["bounds"])
bboxImg = box(*img.bounds)
bboxArea.intersection(bboxImg)

In [ ]:
sm = rio.mask.mask(img, [bboxArea], all_touched=True, crop=True)
profile["height"] = sm[0].shape[1]
profile["width"] = sm[0].shape[2]
profile["transform"] = sm[1]
profile["dtype"] = rio.float32

In [ ]:
import matplotlib.pyplot as plt  # plotting tools
import numpy as np

In [ ]:
sm[0][band].mean()

In [ ]:
np.nanmean(sm[0][band])

In [ ]:
np.nanstd(sm[0][band], axis=(0,1))

In [ ]:
sm[0][band].std()

In [ ]:
image_normalize(sm[0][band]).astype(profile["dtype"]).sum()

In [ ]:
with rio.open(config.out_image_dir / f"test2.png", "w", **profile) as dst:
    for band in range(len(config.bands)):
        norm_band = image_normalize(sm[0][band]).astype(profile["dtype"])
        dst.write(norm_band, band + 1)